In [12]:
!pip install beautifulsoup4
!pip install requests
!pip install requests_html

In [13]:
from bs4 import BeautifulSoup
from requests_html import HTMLSession

import pandas as pd

ImportError: dlopen(/Users/alexsmith/Programming/Projects/ticket-predictor/.venv/lib/python3.10/site-packages/lxml/etree.cpython-310-darwin.so, 0x0002): symbol not found in flat namespace (_exsltDateXpathCtxtRegister)

In [16]:
import requests

home_url = 'https://www.livefootballtickets.com/euro-cup-tickets.html'
country_url = 'https://www.livefootballtickets.com/euro-cup/albania-football-tickets.html'
match_url = 'https://www.livefootballtickets.com/fixtures/match-1-germany-v-a2-tickets-euro-2024.html'

html_text = requests.get(country_url).text

print(html_text)



<!DOCTYPE html>
<html class="no-js" lang="en">
    <head><meta charset="utf-8" />
        <meta name="viewport" content="initial-scale=1.0, width=device-width
        <link rel="preconnect dns-prefetch" href="//www.googletagmanager.com" /><link rel="preconnect dns-prefetch" href="//cdnjs.cloudflare.com" /><link rel="preconnect dns-prefetch" href="//widget.trustpilot.com" /><link rel="preload" href="/fonts/dosis-v26-vietnamese_latin-ext_latin-700.woff2" as="font" type="font/woff2" crossorigin="anonymous" /><link rel="preload" href="/fonts/open-sans-v29-vietnamese_latin-ext_latin_hebrew_greek-ext_greek_cyrillic-ext_cyrillic-regular.woff2" as="font" type="font/woff2" crossorigin="anonymous" /><link rel="preload" href="/fonts/open-sans-v29-vietnamese_latin-ext_latin_hebrew_greek-ext_greek_cyrillic-ext_cyrillic-700.woff2" as="font" type="font/woff2" crossorigin="anonymous" />
        
        <style>
            @font-face {
              font-family: 'Dosis';
              font-display: s

In [8]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html_text, 'html.parser')

prices = soup.find_all("div", {"class": "ticket-price"})

In [11]:
tickets = soup.select('li.ticket-row')

for ticket in tickets:
    stadium_id = ticket['data-stadium-id']
    ticket_heading = ticket.find('h5', class_='ticket-heading').text
    ticket_price = ticket.find('div', class_='ticket-price').h4.text
    ticket_quantities = [int(option['value']) for option in ticket.select('select.ticket-quantity option')]
    # highest_quantity = max(ticket_quantities)
    
    print(f"Stadium ID: {stadium_id}")
    print(f"Ticket Heading: {ticket_heading}")
    print(f"Ticket Price: {ticket_price}")
    # print(f"Highest Ticket Quantity: {highest_quantity}")
    print("-----")


Stadium ID: C_1950
Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £ 479.00
-----
Stadium ID: C_1950
Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £ 479.98
-----
Stadium ID: C_1950
Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £ 480.95
-----
Stadium ID: C_1950
Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £ 480.96
-----
Stadium ID: C_1950
Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £ 480.97
-----
Stadium ID: C_1950
Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £ 480.98
-----
Stadium ID: C_1950
Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £ 482.00
-----
Stadium ID: C_1950
Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £ 486.00
-----
Stadium ID: C_1950
Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £486.22
-----
Stadium ID: C_1950
Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £ 487.99
-----
Stadium ID: C_1950
Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket 

In [ ]:
soup = BeautifulSoup(requests.get(home_url).text, 'html.parser')
country_links = soup.select('div.team-list a.team-list-item')

